In [2]:
import pandas as pd

# Read the CSV with semicolon separator
df = pd.read_csv(r'C:\sachin\Python\Final_Code\NLP\labelled_newscatcher_dataset.csv', sep=';')
df

,topic,link,domain,published_date,title,lang
0,SCIENCE,https://www.eurekalert.org/pub_releases/2020-0...,eurekalert.org,2020-08-06 13:59:45,A closer look at water-splitting's solar fuel ...,en
1,SCIENCE,https://www.pulse.ng/news/world/an-irresistibl...,pulse.ng,2020-08-12 15:14:19,"An irresistible scent makes locusts swarm, stu...",en
2,SCIENCE,https://www.express.co.uk/news/science/1322607...,express.co.uk,2020-08-13 21:01:00,Artificial intelligence warning: AI will know ...,en
3,SCIENCE,https://www.ndtv.com/world-news/glaciers-could...,ndtv.com,2020-08-03 22:18:26,Glaciers Could Have Sculpted Mars Valleys: Study,en
4,SCIENCE,https://www.thesun.ie/tech/5742187/perseid-met...,thesun.ie,2020-08-12 19:54:36,Perseid meteor shower 2020: What time and how ...,en
...,...,...,...,...,...,...
108769,NATION,https://www.vanguardngr.com/2020/08/pdp-govern...,vanguardngr.com,2020-08-08 02:40:00,PDP governors’ forum urges security agencies t...,en
108770,BUSINESS,https://www.patentlyapple.com/patently-apple/2...,patentlyapple.com,2020-08-08 01:27:12,"In Q2-20, Apple Dominated the Premium Smartpho...",en
108771,HEALTH,https://www.belfastlive.co.uk/news/health/coro...,belfastlive.co.uk,2020-08-12 17:01:00,Coronavirus Northern Ireland: Full breakdown s...,en
108772,ENTERTAINMENT,https://www.thenews.com.pk/latest/696364-paul-...,thenews.com.pk,2020-08-05 04:59:00,Paul McCartney details post-Beatles distress a...,en


In [3]:
df.drop(columns = ['link','domain','published_date','lang'], inplace=True) 

In [4]:
df 

,topic,title
0,SCIENCE,A closer look at water-splitting's solar fuel ...
1,SCIENCE,"An irresistible scent makes locusts swarm, stu..."
2,SCIENCE,Artificial intelligence warning: AI will know ...
3,SCIENCE,Glaciers Could Have Sculpted Mars Valleys: Study
4,SCIENCE,Perseid meteor shower 2020: What time and how ...
...,...,...
108769,NATION,PDP governors’ forum urges security agencies t...
108770,BUSINESS,"In Q2-20, Apple Dominated the Premium Smartpho..."
108771,HEALTH,Coronavirus Northern Ireland: Full breakdown s...
108772,ENTERTAINMENT,Paul McCartney details post-Beatles distress a...


#### Check the missing values 

In [5]:
df.isna().sum()

topic    0
title    0
dtype: int64

In [6]:
df.isnull().sum()

topic    0
title    0
dtype: int64

In [7]:
df.nunique()

topic         8
title    103180
dtype: int64

In [8]:
df['topic'].unique()

array(['SCIENCE', 'TECHNOLOGY', 'HEALTH', 'WORLD', 'ENTERTAINMENT',
       'SPORTS', 'BUSINESS', 'NATION'], dtype=object)

In [9]:
df['topic'].value_counts()

topic
TECHNOLOGY       15000
HEALTH           15000
WORLD            15000
ENTERTAINMENT    15000
SPORTS           15000
BUSINESS         15000
NATION           15000
SCIENCE           3774
Name: count, dtype: int64

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108774 entries, 0 to 108773
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   topic   108774 non-null  object
 1   title   108774 non-null  object
dtypes: object(2)
memory usage: 1.7+ MB


### Preprocess the data 

In [11]:
import nltk
import string
import re
import inflect
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

In [12]:
nltk.download('punkt',force = True)
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to C:\Users\Sachin
[nltk_data]     Hembram\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to C:\Users\Sachin
[nltk_data]     Hembram\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Sachin
[nltk_data]     Hembram\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [13]:
df

,topic,title
0,SCIENCE,A closer look at water-splitting's solar fuel ...
1,SCIENCE,"An irresistible scent makes locusts swarm, stu..."
2,SCIENCE,Artificial intelligence warning: AI will know ...
3,SCIENCE,Glaciers Could Have Sculpted Mars Valleys: Study
4,SCIENCE,Perseid meteor shower 2020: What time and how ...
...,...,...
108769,NATION,PDP governors’ forum urges security agencies t...
108770,BUSINESS,"In Q2-20, Apple Dominated the Premium Smartpho..."
108771,HEALTH,Coronavirus Northern Ireland: Full breakdown s...
108772,ENTERTAINMENT,Paul McCartney details post-Beatles distress a...


#### Remove URL

In [14]:
# Define function to remove URLs from a text string
def clean_url(text):
    return re.sub(r'https?://\S+', '', str(text))

# Apply the function to the 'details' column
df['title'] = df['title'].apply(clean_url)


#### Remove punctuaftion

In [15]:
def remove_punctuation(text):
    text = str(text)
    return re.sub(rf"[{re.escape(string.punctuation)}]", "", text)

In [16]:
df['title'] = df['title'].apply(remove_punctuation)

#### Convert into lowercases 

In [17]:
def to_lowercase(text):
    return str(text).lower()

df['title'] = df['title'].apply(to_lowercase)

In [18]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to C:\Users\Sachin
[nltk_data]     Hembram\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to C:\Users\Sachin
[nltk_data]     Hembram\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [19]:
def clean_tokenization(text):
    if isinstance(text, str):
        return word_tokenize(text)
    else:
        return []  # or return text

df['title'] = df['title'].apply(clean_tokenization)

In [20]:
import nltk

# For stopwords
nltk.download('stopwords')

# For lemmatization  
nltk.download('wordnet')
nltk.download('omw-1.4')

# For better lemmatization (POS tagging)
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package stopwords to C:\Users\Sachin
[nltk_data]     Hembram\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Sachin
[nltk_data]     Hembram\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Sachin
[nltk_data]     Hembram\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Sachin Hembram\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\Sachin Hembram\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [21]:
nltk.download("all")

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to C:\Users\Sachin
[nltk_data]    |     Hembram\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to C:\Users\Sachin
[nltk_data]    |     Hembram\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\Sachin
[nltk_data]    |     Hembram\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     C:\Users\Sachin
[nltk_data]    |     Hembram\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_t

True

In [22]:
!pip install --upgrade nltk

Defaulting to user installation because normal site-packages is not writeable


In [23]:
from nltk.stem import PorterStemmer
Stemmer = PorterStemmer()

In [24]:
def clean_stem(token):
  return[Stemmer.stem(i) for i in token]
df['title'] = df['title'].apply(clean_stem)

#### removing stopwords 

In [25]:
from nltk.corpus import stopwords
# Get English stopwords
stopwords.words('english')

['a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 "he'd",
 "he'll",
 'her',
 'here',
 'hers',
 'herself',
 "he's",
 'him',
 'himself',
 'his',
 'how',
 'i',
 "i'd",
 'if',
 "i'll",
 "i'm",
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it'd",
 "it'll",
 "it's",
 'its',
 'itself',
 "i've",
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'on

In [26]:
stop_words = set(stopwords.words('english'))
def clean_stopwords(token):
    return[item for item in token if item not in stop_words]
df['title'] = df['title'].apply(clean_stopwords)

#### Label encoding 

In [27]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['topic'] = le.fit_transform(df['topic'])  # Save encoded labels


#### Train test split 

In [28]:
from sklearn.model_selection import train_test_split

# Step 1: Define input (X) and target (y)
X = df['title']                # Input: cleaned text
y = df['topic']                # Target: encoded topic labels

# Step 2: Split into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,             # 20% for testing
    stratify=y,                # Keeps class distribution same in both sets
    random_state=42            # Ensures reproducibility
)

#### Model Building 

In [29]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [30]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow import keras

# Load IMDB dataset
(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data(num_words=10000)

# Pad sequences to make inputs uniform
max_length = 200
X_train = pad_sequences(X_train, maxlen=max_length)
X_test = pad_sequences(X_test, maxlen=max_length)

#### Define the LSTM Model 

In [ ]:
# model = keras.Sequential([
#     keras.layers.Embedding(input_dim=10000, output_dim=32, input_length=max_length),
#     keras.layers.LSTM(64, return_sequences=True),
#     keras.layers.LSTM(32),
#     keras.layers.Dense(1, activation='sigmoid')
# ])

C:\Users\Sachin Hembram\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
from tensorflow import keras

vocab_size = 10000
max_length = 100  
num_classes = 7

model = keras.Sequential([
    keras.layers.Embedding(input_dim=vocab_size, output_dim=32, input_length=max_length),
    keras.layers.LSTM(64, return_sequences=True),
    keras.layers.LSTM(32),
    keras.layers.Dense(num_classes, activation='softmax')
])

# Build the model
model.build(input_shape=(None, max_length))

#### Compile the model 

In [42]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [43]:
model.fit(X_train, y_train, epochs=10, batch_size=32, shuffle=True, verbose=1)

Epoch 1/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 72s 89ms/step - accuracy: 0.5614 - loss: 0.6894
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 68s 87ms/step - accuracy: 0.8638 - loss: 0.3273
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 68s 87ms/step - accuracy: 0.9220 - loss: 0.2066
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 68s 87ms/step - accuracy: 0.9484 - loss: 0.1475
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 67s 86ms/step - accuracy: 0.9667 - loss: 0.1022
Epoch 6/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 754s 965ms/step - accuracy: 0.9777 - loss: 0.0750
Epoch 7/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 74s 94ms/step - accuracy: 0.9829 - loss: 0.0576
Epoch 8/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 69s 88ms/step - accuracy: 0.9884 - loss: 0.0429
Epoch 9/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 70s 89ms/step - accuracy: 0.9900 - loss: 0.0354
Epoch 10/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 96s 123ms/step - accuracy: 0.9846 - loss: 0.0465


#### BiLSTM

In [44]:
X = df['title']                
y = df['topic']                

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

In [45]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Parameters
max_words = 10000  # max vocabulary size
max_len = 200      # max sequence length

# Tokenizer
tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

# Convert text to sequences
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post', truncating='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding='post', truncating='post')

# Vocabulary size
vocab_size = len(tokenizer.word_index) + 1


In [46]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Encode labels
label_encoder = LabelEncoder()
y_train_enc = label_encoder.fit_transform(y_train)
y_test_enc = label_encoder.transform(y_test)

# One-hot encoding
y_train_cat = to_categorical(y_train_enc)
y_test_cat = to_categorical(y_test_enc)

# Number of classes
num_classes = y_train_cat.shape[1]


In [47]:
y.value_counts()

topic
6    15000
2    15000
7    15000
1    15000
5    15000
0    15000
3    15000
4     3774
Name: count, dtype: int64

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout

embedding_dim = 64

model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len),
    Bidirectional(LSTM(64, return_sequences=True)),
    Dropout(0.5),
    Bidirectional(LSTM(32)),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])

C:\Users\Sachin Hembram\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [49]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [50]:
model.fit(
    X_train_pad, y_train_cat,
    validation_data=(X_test_pad, y_test_cat),
    epochs=10,
    batch_size=64
)   

Epoch 1/10
1360/1360 ━━━━━━━━━━━━━━━━━━━━ 360s 259ms/step - accuracy: 0.6319 - loss: 1.0232 - val_accuracy: 0.7515 - val_loss: 0.7280
Epoch 2/10
1360/1360 ━━━━━━━━━━━━━━━━━━━━ 357s 262ms/step - accuracy: 0.7889 - loss: 0.6506 - val_accuracy: 0.7868 - val_loss: 0.6448
Epoch 3/10
1360/1360 ━━━━━━━━━━━━━━━━━━━━ 417s 307ms/step - accuracy: 0.8276 - loss: 0.5430 - val_accuracy: 0.7892 - val_loss: 0.6431
Epoch 4/10
1360/1360 ━━━━━━━━━━━━━━━━━━━━ 455s 334ms/step - accuracy: 0.8477 - loss: 0.4757 - val_accuracy: 0.7868 - val_loss: 0.6587
Epoch 5/10
1360/1360 ━━━━━━━━━━━━━━━━━━━━ 471s 347ms/step - accuracy: 0.8617 - loss: 0.4267 - val_accuracy: 0.7890 - val_loss: 0.6848
Epoch 6/10
1360/1360 ━━━━━━━━━━━━━━━━━━━━ 31391s 23s/step - accuracy: 0.8759 - loss: 0.3825 - val_accuracy: 0.7896 - val_loss: 0.6991
Epoch 7/10
1360/1360 ━━━━━━━━━━━━━━━━━━━━ 304s 223ms/step - accuracy: 0.8858 - loss: 0.3471 - val_accuracy: 0.7876 - val_loss: 0.7481
Epoch 8/10
1360/1360 ━━━━━━━━━━━━━━━━━━━━ 318s 234ms/step - ac

In [51]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ (None, 200, 64)        │     2,880,768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 200, 128)       │        66,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 200, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 64)             │        41,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 8)              │           520 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,978,138 (34.25 MB)

 Trainable params: 2,992,712 (11.42 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 5,985,426 (22.83 MB)

In [52]:
import pickle

with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)


In [53]:
label_mapping = {
    0: 'BUSINESS',
    1: 'ENTERTAINMENT',
    2: 'HEALTH',
    3: 'NATION',
    4: 'SCIENCE',
    5: 'SPORTS',
    6: 'TECHNOLOGY',
    7: 'WORLD'
}
